In [29]:
%%HTML
<h1 align ="center">Computer Systems</h1>
<h2 align="center"> Predicting Game Trends and Sales</h2>
<h3 align="center"> Ammar Rashed <br /><br />   Abdul Jaleel Yusif<br /> <br /> Muhammed Emre Sevuk <br /> <br /> Ersin Onur Erdogan </h3>
<p>With this work, we analyze the game datas from previous years and based on them predicting the good game opportunities in the coming years.</p>
<p>To be able to do that, we found most succesful platforms, most succesful publishers and most succesful game genres. </p>

In [18]:

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_1fdbb9571d5043db94db4dbded7cbfb1(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '79361bf32ffa4e9aad00318f9a848e64')
    hconf.set(prefix + '.username', '624ea908149b472d88ecb4692f51b5a6')
    hconf.set(prefix + '.password', 'M-r0(^u&Tm!s7c7J')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_1fdbb9571d5043db94db4dbded7cbfb1(name)

df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://Project1." + name + "/vgsales.csv")
df.show(5)


+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports| Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing| Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports| Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing| Nintendo|   11.27|    8.89|   10.22|        1.0|       31.37|
+----+--------------------+--------+----+------------+---------+

In [2]:
%%HTML
<p> In order to find on which platform we shall create games to be able to sale it more, we tried to find the answer of </br> " What is the most Successful Platform for us ?"</p>

In [19]:
# Most Successful platform
df.select(df["Platform"], df["Global_Sales"]).groupBy("Platform").sum().orderBy('sum(Global_Sales)', ascending=False).show(5)

+--------+------------------+
|Platform| sum(Global_Sales)|
+--------+------------------+
|     PS2|1255.6400000000078|
|    X360| 979.9600000000025|
|     PS3| 957.8400000000026|
|     Wii| 926.7100000000017|
|      DS| 822.4900000000005|
+--------+------------------+
only showing top 5 rows



In [3]:
%%HTML
<p> After finding the best platform, we focused to find the best genre for the choosen platform</p>

In [21]:
# Best Genre of Best Platform
best_game = df.where(df['Platform'] == "PS2").orderBy('Global_Sales', ascending=False)
best_game.show()
best_genre = best_game.select('Genre').groupBy("Genre").count().orderBy('count', ascending=False).show(5)

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|  18|Grand Theft Auto:...|     PS2|2004|      Action|Take-Two Interactive|    9.43|     0.4|    0.41|      10.57|       20.81|
|  25|Grand Theft Auto:...|     PS2|2002|      Action|Take-Two Interactive|    8.41|    5.49|    0.47|       1.78|       16.15|
|  29|Gran Turismo 3: A...|     PS2|2001|      Racing|Sony Computer Ent...|    6.85|    5.09|    1.87|       1.16|       14.98|
|  39|Grand Theft Auto III|     PS2|2001|      Action|Take-Two Interactive|    6.99|    4.51|     0.3|        1.3|        13.1|
|  48|      Gran Turismo 4|     PS2|2004|      Racing|Sony Computer Ent...|    3.01|    0.01|     1.1|  

In [4]:
%%HTML
<p> When we found the best platform and genre for this platform, we looked for the best publisher </br> 
and best publishers best genres in order to imitiate their strategies.</p>

In [22]:
#Question 7(a) - Best publisher
df.select(df["Publisher"], df["Global_Sales"]).groupBy("Publisher").sum().orderBy('sum(Global_Sales)', ascending=False).show(5)

#Question 7(b) - Best publisher's genre
pubs_best_game = df.where(df['Publisher'] == "Nintendo").orderBy('Global_Sales', ascending=False)
pubs_best_game.show()
best_genre = pubs_best_game.select('Genre').groupBy("Genre").count().orderBy('count', ascending=False).show(5)

+--------------------+------------------+
|           Publisher| sum(Global_Sales)|
+--------------------+------------------+
|            Nintendo|1786.5600000000013|
|     Electronic Arts|1110.3199999999983|
|          Activision| 727.4600000000003|
|Sony Computer Ent...| 607.5000000000002|
|             Ubisoft|474.71999999999986|
+--------------------+------------------+
only showing top 5 rows

+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports| Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      

In [7]:
%%HTML
<p>Some more additional information about the market </br> <a font = "bold">--- Year with best sales ---</a></p>

In [23]:
# Year with best sales
df.select(df["Year"], df["Global_Sales"]).groupBy("Year").sum().orderBy('sum(Global_Sales)', ascending=False).show(5)

+----+-----------------+
|Year|sum(Global_Sales)|
+----+-----------------+
|2008|678.9000000000023|
|2009|667.3000000000015|
|2007|611.1300000000006|
|2010|600.4500000000019|
|2006|521.0400000000002|
+----+-----------------+
only showing top 5 rows



In [8]:
%%HTML
<p><a font = "bold">--- Which genre has how many games ---</a></p>

In [25]:
# Which genre has how many games

list_of_genres = ["Action","Role-Playing","Puzzle","Shooter","Misc","Fighting","Sports",\
                  "Platform","Strategy","Adventure","Simulation","Racing"]
dict_of_genres = {}


df = df.withColumn("Global_Sales", df["Global_Sales"].cast("float"))
df = df.withColumn("NA_Sales", df["NA_Sales"].cast("float"))
df = df.withColumn("JP_Sales", df["JP_Sales"].cast("float"))
df = df.withColumn("EU_Sales", df["EU_Sales"].cast("float"))
df = df.withColumn("Other_Sales", df["Other_Sales"].cast("float"))
df = df.withColumn("Rank", df["Rank"].cast("integer"))

for i in list_of_genres:
    p = [df.filter(df["Genre"]==i).count()]
    dict_of_genres.setdefault(i, p)
print dict_of_genres

{'Role-Playing': [1488], 'Puzzle': [582], 'Shooter': [1310], 'Misc': [1739], 'Fighting': [848], 'Sports': [2346], 'Platform': [886], 'Adventure': [1286], 'Action': [3316], 'Simulation': [867], 'Strategy': [681], 'Racing': [1249]}


In [9]:
%%HTML
<p><a font = "bold">--- Which Genre is the best to make money ---</a></p>

In [26]:
# Which Genre is the best to make money
dict_of_genres2 = dict_of_genres

df_gb = df.select("Genre", "Global_Sales").groupBy("Genre").sum("Global_Sales").show(5)

+------------+------------------+
|       Genre| sum(Global_Sales)|
+------------+------------------+
|      Racing| 732.0399981327355|
|    Strategy|175.11999987624586|
|    Platform| 831.3700044620782|
|Role-Playing| 927.3700027912855|
|      Sports|1330.9299955032766|
+------------+------------------+
only showing top 5 rows

